# Additional End of week Exercise - week 2

A python learning assistant with tool calling capability and streaming with model toggle between Openai and Google

In [ ]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [ ]:
# constants    
MODEL_GPT = 'gpt-4o-mini'
MODEL_GEMINI = 'gemini-2.5-flash'



In [ ]:
# set up environment
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key and openai_api_key.startswith('sk-proj-'):
    print("Openai API key is set and ready")
else:
    print("Check env file to make sure that openai key is set!")

if google_api_key:
    print(f"Google API Key exists and ready")
else:
    print("Google API Key not set")


In [ ]:
# Connect to Client Liberary
openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

#system prompt
system_prompt = """
You are a helpful programming assistant. When given a technical question about Python, your task is to analyze the question and provide a detailed explanation, formatted in Markdown. Your responses should be clear, concise, and educational, aiming to help the user understand the concept or code in question. Use code blocks, bullet points, and examples as appropriate to illustrate your explanation.
But When you call a tool, limit the response to the tool's response and do not provide any further explanation on the topic.
"""

In [ ]:
DB = "scores.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS scores (
            topic TEXT COLLATE NOCASE PRIMARY KEY,
            score INT NOT NULL
        )
    ''')
    conn.commit()

In [ ]:
def set_score(topic, score):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO scores (topic, score) VALUES (?, ?) ON CONFLICT(topic) DO UPDATE SET score=excluded.score', (topic.lower(), score))
        conn.commit()

In [ ]:
sample_scores = {
    "Python Functions": 95,
    "List Comprehensions": 88,
    "SQLite Usage": 92,
    "Decorators": 85,
    "Exception Handling": 90
}

for topic, score in sample_scores.items():
    set_score(topic, score)

In [ ]:
# get note

def get_score(topic):
    print(f"DATABASE TOOL CALLED: Getting score for {topic}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT score FROM scores WHERE topic = ?', (topic.lower(),))
        result = cursor.fetchone()
        # Return clear note content so the model can use it in the final answer
        return result[0] if result else "No note data available for this topic"

In [ ]:
# Tool Description
scores_function = {
    "name": "get_score",
    "description": "Get the score for a provided topic",
    "parameters": {
        "type": "object",
        "properties": {
            "topic_name": {
                "type": "string",
                "description": "The name of the topic the user wants a score for",
            },
        },
        "required": ["topic_name"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": scores_function}]

In [ ]:
# Tool calling handler

def handle_tool_call(message):
  """message: dict with 'tool_calls' (list of {id, function: {name, arguments}})."""
  responses = []
  for tool_call in message.get("tool_calls", []):
      fn = tool_call.get("function", {})
      if fn.get("name") == "get_score":
          arguments = json.loads(fn.get("arguments", "{}"))
          topic_name = arguments.get("topic_name")
          topic_score = get_score(topic_name)
          responses.append({
              "role": "tool",
              "content": str(topic_score),
              "tool_call_id": tool_call["id"]
          })
          print(responses)
  return responses

In [ ]:
# Shared: streaming + tool-calling loop (OpenAI-compatible client: openai or gemini)
def _history_to_messages(history):
    """Normalize Gradio history to list of {role, content}. Supports 'messages' format or 'tuples' format."""
    if not history:
        return []
    first = history[0]
    if isinstance(first, dict) and "role" in first:
        return [{"role": h["role"], "content": h["content"] if isinstance(h.get("content"), str) else str(h.get("content", ""))} for h in history]
    out = []
    for pair in history:
        u, a = (pair[0], pair[1]) if len(pair) >= 2 else (pair[0], "")
        out.append({"role": "user", "content": u if isinstance(u, str) else str(u)})
        out.append({"role": "assistant", "content": a if isinstance(a, str) else str(a)})
    return out

def chat_stream_with_tools(client, model_name, message, history):
    """Stream chat with tool calling. Yields accumulated content; runs tools and re-requests until model returns text only."""
    history_msgs = _history_to_messages(history)
    messages = [{"role": "system", "content": system_prompt}] + history_msgs + [{"role": "user", "content": message}]

    while True:
        stream = client.chat.completions.create(
            model=model_name,
            messages=messages,
            tools=tools,
            stream=True,
        )
        content = ""
        tool_calls_accum = {}
        finish_reason = None

        for chunk in stream:
            if not chunk.choices:
                continue
            d = chunk.choices[0].delta
            if d.content:
                content += d.content
                yield content
            for tc in d.tool_calls or []:
                i = tc.index
                if i not in tool_calls_accum:
                    tool_calls_accum[i] = {"id": "", "name": "", "arguments": ""}
                if tc.id:
                    tool_calls_accum[i]["id"] = tc.id
                if tc.function and tc.function.name:
                    tool_calls_accum[i]["name"] = tc.function.name
                if tc.function and tc.function.arguments:
                    tool_calls_accum[i]["arguments"] += tc.function.arguments or ""
            finish_reason = chunk.choices[0].finish_reason or finish_reason

        if finish_reason != "tool_calls" or not tool_calls_accum:
            break

        sorted_tcs = [tool_calls_accum[i] for i in sorted(tool_calls_accum)]
        assistant_msg = {
            "role": "assistant",
            "content": content or None,
            "tool_calls": [
                {"id": t["id"], "type": "function", "function": {"name": t["name"], "arguments": t["arguments"]}}
                for t in sorted_tcs
            ],
        }
        messages.append(assistant_msg)
        responses = handle_tool_call(assistant_msg)
        messages.extend(responses)

In [ ]:
# declare the function for openai (streaming + tool calling)
def chat_openai(message, history):
    """Stream response with tool calling via shared logic."""
    yield from chat_stream_with_tools(openai, MODEL_GPT, message, history)

In [ ]:
# declare the function for gemini (streaming + tool calling, same as OpenAI)
def chat_gemini(message, history):
    """Stream response with tool calling via shared logic."""
    yield from chat_stream_with_tools(gemini, MODEL_GEMINI, message, history)

In [ ]:
# model selector: receives message, history (from ChatInterface), and model (from dropdown)
def model_selector(message, history, model):
    if model == "GPT":
        yield from chat_openai(message, history)
    elif model == "Gemini":
        yield from chat_gemini(message, history)
    else:
        raise ValueError("Unknown model")

In [ ]:
# Build chat interface using gr.ChatInterface — handles message, history, and model automatically
model_dropdown = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")

view = gr.ChatInterface(
    fn=model_selector,
    type="messages",
    title="Personal Assistant",
    description="Ask me anything about Python programming. Try: 'What do your notes say about Decorators?'",
    additional_inputs=[model_dropdown],
    examples=[
        ["Explain how enumerate() works in Python", "GPT"],
        ["What is a list comprehension?", "Gemini"],
        ["What is the score for Decorators?", "GPT"],
    ],
)

view.launch()
